In [1]:
import numpy as np
import xarray as xr

In [2]:
coordsA = {'A': ['bad', 'good']}
coordsF = {'F': ['unfit', 'fit']}
coordsR = {'R': ['muddy', 'dry']}
coordsT = {'T': ['poor', 'average', 'top']}
coordsQ = {'Q': ['fail', 'qualified']}
pF = xr.DataArray([0.7, 0.3], dims=('F'), coords=coordsF)
pA_F = xr.DataArray([[0.95, 0.05], [0.2, 0.8]], dims=('F', 'A'), coords={**coordsA, **coordsF})
pR = xr.DataArray([0.6, 0.4], dims=('R'), coords=coordsR)
pT_RF = xr.DataArray([[[0.3, 0.4, 0.3], [0.7, 0.25, 0.05]], [[0.02, 0.08, 0.90], [0.2, 0.3, 0.5]]], dims=('F', 'R', 'T'), coords={**coordsT, **coordsR, **coordsF}) # times/groups are in "reverse" order compared to Koller
pQ_T = xr.DataArray([[0.99, 0.01], [0.4, 0.6], [0.1, 0.9]], dims=('T', 'Q'), coords={**coordsT, **coordsQ}) # here it changes the order of the 3 rows

def normalize(a):
    return a / a.sum()

# derived joint probabilities for the exercices
pAF = pF * pA_F
pAFT = (pAF * pR * pT_RF).sum(dim='R')
p_all = pR*pF*pT_RF*pA_F*pQ_T

In [3]:
p_all.sum()

<xarray.DataArray ()>
array(1.)

# Simple 2 variable case

In [4]:
pAF # joint distribution

<xarray.DataArray (F: 2, A: 2)>
array([[0.665, 0.035],
       [0.06 , 0.24 ]])
Coordinates:
  * F        (F) <U5 'unfit' 'fit'
  * A        (A) <U4 'bad' 'good'

In [5]:
# Q1
# proba of being fit, sum rule
print("A proportion of", pAF.sum(dim='A').sel(F='fit').values, "is fit")
# but we also have all values p(A)
pAF.sum(dim='A')

A proportion of 0.3 is fit


<xarray.DataArray (F: 2)>
array([0.7, 0.3])
Coordinates:
  * F        (F) <U5 'unfit' 'fit'

In [6]:
# Q2
# proba of being fit, knowing we did good at the aerobic test
# conditioning (taking just a value) and renormalizing
print("Given A=good, the proba of being fit is", normalize(pAF.sel(A='good')).sel(F='fit').values)
# but we also have all values p(F|A=good)
normalize(pAF.sel(A='good'))

Given A=good, the proba of being fit is 0.8727272727272728


<xarray.DataArray (F: 2)>
array([0.12727273, 0.87272727])
Coordinates:
  * F        (F) <U5 'unfit' 'fit'
    A        <U4 'good'

In [7]:
# exploring
normalize(pAF.sel(A='bad'))

<xarray.DataArray (F: 2)>
array([0.91724138, 0.08275862])
Coordinates:
  * F        (F) <U5 'unfit' 'fit'
    A        <U4 'bad'

In [8]:
# more exploration
pAF.sum(dim='F')

<xarray.DataArray (A: 2)>
array([0.725, 0.275])
Coordinates:
  * A        (A) <U4 'bad' 'good'

# Three variables

In [9]:
print("shape:", pAFT.shape)
print("total size:", pAFT.size)
print(pAFT.sum().values)

pAFT

shape: (2, 2, 3)
total size: 12
0.9999999999999999


<xarray.DataArray (F: 2, A: 2, T: 3)>
array([[[0.3059 , 0.2261 , 0.133  ],
        [0.0161 , 0.0119 , 0.007  ]],

       [[0.00552, 0.01008, 0.0444 ],
        [0.02208, 0.04032, 0.1776 ]]])
Coordinates:
  * F        (F) <U5 'unfit' 'fit'
  * A        (A) <U4 'bad' 'good'
  * T        (T) <U7 'poor' 'average' 'top'

In [10]:
# Q3
# knowing there was a good test result A=good,
# what is the probability of getting a time in the top, T=top?
print("Given A=good, prob(T=top) =", normalize(pAFT.sel(A='good').sum(dim='F')).sel(T='top').values)
# the distribution (on T)
normalize(pAFT.sel(A='good').sum(dim='F'))

Given A=good, prob(T=top) = 0.6712727272727274


<xarray.DataArray (T: 3)>
array([0.13883636, 0.18989091, 0.67127273])
Coordinates:
    A        <U4 'good'
  * T        (T) <U7 'poor' 'average' 'top'

In [11]:
# let's check what we get with a bad test result
normalize(pAFT.sel(A='bad').sum(dim='F'))

<xarray.DataArray (T: 3)>
array([0.42954483, 0.32576552, 0.24468966])
Coordinates:
    A        <U4 'bad'
  * T        (T) <U7 'poor' 'average' 'top'

In [12]:
# can we be fit but fail tests and races
normalize(pAFT.sel(A='bad', T='poor')) # ... not so much

<xarray.DataArray (F: 2)>
array([0.98227474, 0.01772526])
Coordinates:
  * F        (F) <U5 'unfit' 'fit'
    A        <U4 'bad'
    T        <U7 'poor'

In [13]:
# if we know we're fit, what is the probabily to perform at the race? 
normalize(pAFT.sel(F='fit').sum('A'))

<xarray.DataArray (T: 3)>
array([0.092, 0.168, 0.74 ])
Coordinates:
    F        <U5 'fit'
  * T        (T) <U7 'poor' 'average' 'top'

In [14]:
# same, but knowing a test results
normalize(pAFT.sel(A='bad', F='fit'))
# (no change, so there is a conditional independance)
# (T seems indep of A, given F=fit)
# (already knowing F=fit, knowing A does not bring information on T)

<xarray.DataArray (T: 3)>
array([0.092, 0.168, 0.74 ])
Coordinates:
    F        <U5 'fit'
    A        <U4 'bad'
  * T        (T) <U7 'poor' 'average' 'top'

In [15]:
# same with F='unfit'
normalize(pAFT.sel(A='bad', F='unfit')) / normalize(pAFT.sel(F='unfit').sum('A'))
# ... the ratio p(T|A=bad,F=unfit) / p(T|F=unfit) is 1 for all T

<xarray.DataArray (T: 3)>
array([1., 1., 1.])
Coordinates:
    F        <U5 'unfit'
    A        <U4 'bad'
  * T        (T) <U7 'poor' 'average' 'top'

# All variables

In [16]:
print("shape:", p_all.shape)
print("total size:", p_all.size)
print(p_all.sum().values)

shape: (2, 2, 3, 2, 2)
total size: 48
1.0


In [17]:
p_all

<xarray.DataArray (R: 2, F: 2, T: 3, A: 2, Q: 2)>
array([[[[[1.18503e-01, 1.19700e-03],
          [6.23700e-03, 6.30000e-05]],

         [[6.38400e-02, 9.57600e-02],
          [3.36000e-03, 5.04000e-03]],

         [[1.19700e-02, 1.07730e-01],
          [6.30000e-04, 5.67000e-03]]],


        [[[7.12800e-04, 7.20000e-06],
          [2.85120e-03, 2.88000e-05]],

         [[1.15200e-03, 1.72800e-03],
          [4.60800e-03, 6.91200e-03]],

         [[3.24000e-03, 2.91600e-02],
          [1.29600e-02, 1.16640e-01]]]],



       [[[[1.84338e-01, 1.86200e-03],
          [9.70200e-03, 9.80000e-05]],

         [[2.66000e-02, 3.99000e-02],
          [1.40000e-03, 2.10000e-03]],

         [[1.33000e-03, 1.19700e-02],
          [7.00000e-05, 6.30000e-04]]],


        [[[4.75200e-03, 4.80000e-05],
          [1.90080e-02, 1.92000e-04]],

         [[2.88000e-03, 4.32000e-03],
          [1.15200e-02, 1.72800e-02]],

         [[1.20000e-03, 1.08000e-02],
          [4.80000e-03, 4.32000e-02]]]]])
Coordinates:
  * R        (R) <U5 'muddy' 'dry'
  * F        (F) <U5 'unfit' 'fit'
  * T        (T) <U7 'poor' 'average' 'top'
  * A        (A) <U4 'bad' 'good'
  * Q        (Q) <U9 'fail' 'qualified'

In [18]:
# Q4
# probability of getting qualified, given a good aerobic test (and nothing else)
normalize(p_all.sel(A='good').sum(dim=('R', 'F', 'T')))

<xarray.DataArray (Q: 2)>
array([0.28053164, 0.71946836])
Coordinates:
    A        <U4 'good'
  * Q        (Q) <U9 'fail' 'qualified'

In [19]:
# it is still low, so let's check the probability with a bad test result
normalize(p_all.sel(A='bad').sum(dim=('R', 'F', 'T')))

<xarray.DataArray (Q: 2)>
array([0.58002455, 0.41997545])
Coordinates:
    A        <U4 'bad'
  * Q        (Q) <U9 'fail' 'qualified'

In [20]:
# and the "prior" probability with no information
p_all.sum(dim=('A', 'R', 'F', 'T'))

<xarray.DataArray (Q: 2)>
array([0.497664, 0.502336])
Coordinates:
  * Q        (Q) <U9 'fail' 'qualified'